In [ ]:
import sys

sys.path.insert(0, '../03_Remake_with_TDD/algorithms')
from ht_time import *

sys.path.insert(0, '../03_Remake_with_TDD/postgresql')
from ht_match_postgresql import *

In [ ]:
ht_time = HattrickTime()
folder = ht_time.findLastSunOrWed()
# folder = '2018/04/08'
print("folder =", folder)

In [ ]:
import re
from bs4 import BeautifulSoup

In [ ]:
import sys

sys.path.insert(0, '../97_Parse/')
from parseMatch import *

sys.path.insert(0, '../98_Crawler/01_DualMonitor/')
from DualMonitor import getIsDualMonitor

sys.path.insert(0, '../98_Crawler/02_WebBrowser/')
from FirefoxTab import clickTabOfFireFox
from crawMatch import *

sys.path.insert(0, '../98_Crawler/03_OnScreen/')
from Empty import clickEmpty
from MoveScroll import moveScroll
from MoveFindMoveClick import moveByImage, moveByImageWithModify

sys.path.insert(0, '../98_Crawler/04_Login/')
from HattrickLogInOut import clickHattrickLogin, clickHattrickLogout

In [ ]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

clickTabOfFireFox(4, isDualMonitor)

clickEmpty(305, isDualMonitor)
moveScroll(20)
clickHattrickLogin(500, 305, isDualMonitor)

# clickHattrickLogout(1270, 370, isDualMonitor)

In [ ]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

# Hattrick HeavyDefence Tab
clickTabOfFireFox(4, isDualMonitor)

# # Hattrick Login by Facebook
clickEmpty(305, isDualMonitor)
moveScroll(20)
clickHattrickLogin(500, 305, isDualMonitor)

# Hattrick MyClub
moveScroll(20)
moveByImage(530, 342, 77, 42, isDualMonitor, 'screen-MyClub.png')

# Hattrick Match
clickEmpty(700, isDualMonitor)
moveScroll(20)
moveByImage(384, 698, 80, 108, isDualMonitor, 'screen-Match.png')

# Hattrick LastMatch
clickEmpty(831, isDualMonitor)
moveScroll(20)
# moveByImageWithModify(1040, 831, 15, -15, 58, 48, isDualMonitor, 'screen-LastMatch.png')
moveAndClick(1082, 841, isDualMonitor)
time.sleep(5)

# FirefoxFile
moveAndClick(90, 15, isDualMonitor)

# SavePageAs
time.sleep(3)
moveAndClick(91, 130, isDualMonitor)

# Type match
time.sleep(3)
typeFileName()

# rwoo folder
moveAndClick(570, 460, isDualMonitor)

# 02_workspace folder
moveAndClicks(720, 440, isDualMonitor, 2)

# 04_Hattrick folder
moveAndClicks(720, 500, isDualMonitor, 2)

# HattrickStudy
moveAndClicks(720, 415, isDualMonitor, 2)

# 01_Heavy2Defence
moveAndClicks(720, 440, isDualMonitor, 2)

# 2018
moveAndClicks(720, 440, isDualMonitor, 2)

# Last Month
clickLastMonthOrDay()

# Last Day
clickLastMonthOrDay()

# Save
moveByImage(1280, 847, 91, 41, isDualMonitor, 'screen-Save.png')

# Hattrick Logout by Facebook
clickEmpty(336, isDualMonitor)
moveScroll(20)
clickHattrickLogout(1270, 370, isDualMonitor)

# Jupyter Tab
clickTabOfFireFox(7, isDualMonitor)

In [ ]:
from ht_file import *

In [ ]:
HattrickFile = HattrickFile()
HattrickFile.check_and_delete_folder(folder + '/match_files')

In [ ]:
from ht_match import *

In [ ]:
HattrickMatch = HattrickMatch()
match_dict_list = HattrickMatch.findMatchList(folder + '/match.html', 'mydatabase2')
match_str_list = HattrickMatch.matchDictListToMatchStrList(match_dict_list)

print("\ncat match_str_List\n")
for str_line in match_str_list:
    print(str_line)

In [ ]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

In [ ]:
for line in match_str_list:
    with open (folder + '/match.txt', 'a') as f:
        f.write(line + "\n")

In [ ]:
# From match.txt to match table

In [ ]:
import psycopg2

In [ ]:
conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
conn

In [ ]:
# def dropMatchTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """DROP TABLE IF EXISTS match ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
#dropMatchTable()

In [ ]:
# def createMatchTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """
# CREATE TABLE match (
#     Date DATE,          --01
#     Po   VARCHAR(5),    --02 Position
#     Num  INT,           --03 Number
#     RT   NUMERIC(2, 1), --04 Rating
#     sMin INT,           --05 Start Min
#     eMin INT            --06 End Min
# ) ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
# createMatchTable()

In [ ]:
hattrickMatchPostgresql = HattrickMatchPostgreSQL()

In [ ]:
hattrickMatchPostgresql.select_count_of_match(conn)

In [ ]:
import csv
with open(folder + '/match.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if(re.match( '^[0-9]{1,2}', row[1] ) ) :
            hattrickMatchPostgresql.insert_to_match(conn, folder.replace('/', '-'), row)

In [ ]:
hattrickMatchPostgresql.select_count_of_match(conn)

In [ ]:
hattrickMatchPostgresql.update_rt_of_match(conn)

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByDate(folder, conn)
hattrickMatchPostgresql.printMatchByDate(tupleList)

In [ ]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

In [ ]:
with open ( folder + '/match.txt', 'a' ) as f:
    f.write("po,num,rt,sMin,eMin\n")
    f.write( '\n'.join('%s,%s,%s,%s,%s' % myTuple for myTuple in tupleList) + '\n' )

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'KP', [1,2])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [1,2])

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'WB', [44,46,54,58])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [44,46,54,58])

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'CD', [33,43,59,61,64,66])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [33,43,59,61,64,66])

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'W', [7,13,15,48,63])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [7,13,15,48,63])

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'IM', [9,10,11,12,31])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [9,10,11,12,31])

In [ ]:
tupleList = hattrickMatchPostgresql.getMatchByPoNum(conn, 'FW', [16,17,18,60])
hattrickMatchPostgresql.printMatchByPoNum(tupleList, [16,17,18,60])

In [ ]:
conn.close()
print(conn)

In [ ]:
conn = None
print(conn)